In [3]:
%matplotlib inline
import pandas as pd

time: 2.77 ms


In [1]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime
from pyspark import SQLContext
sqlContext = SQLContext(sc)

import pyspark.sql.functions as F
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('Agg')
#import matplotlib.pyplot
import matplotlib.pyplot as plt

/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)
/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [4]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

time: 4.61 ms


In [3]:
cast = pd.DataFrame.from_csv('data/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,NaN
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4
3,Revan,2012,Diego James,actor,Dago,NaN
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12


In [4]:
release_dates = pd.DataFrame.from_csv('data/release_dates.csv', index_col=None,
                                      parse_dates=['date'], infer_datetime_format=True)
release_dates.head()

,title,year,country,date
0,0_1_0,2008,Poland,2008-11-14
1,Ai no Sanka,1967,Japan,1967-01-01
2,A Thousand to One,1920,USA,1920-12-07
3,A Prince of a King,1923,USA,1923-10-13
4,A Prince of a King,1923,Netherlands,1924-08-08


In [6]:
sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true')\
    .load(os.path.expanduser('~/Dropbox/Projects/TreeBoa/pandas/pycon/data/release_dates.csv'))\
    .write.mode('overwrite').parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/release_dates'))

time: 2.54 s


In [7]:
release_dates = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/release_dates'))
release_dates.cache()
cast = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/cast'))
cast.cache()

DataFrame[title: string, year: int, name: string, type: string, character: string, n: int]

time: 232 ms


### Make a bar plot of the months in which movies with "Christmas" in their title tend to be released in the USA.

In [10]:
release_dates[release_dates.title.rlike('Christmas')]\
    .select(F.month(release_dates.date).alias('month'))\
    .groupby('month')\
    .count()\
    .sort('month')\
    .show()

+-----+-----+
|month|count|
+-----+-----+
|    1|   14|
|    2|    5|
|    3|    4|
|    4|    5|
|    5|    8|
|    6|    8|
|    7|    2|
|    8|    7|
|    9|    7|
|   10|   22|
|   11|  175|
|   12|  321|
+-----+-----+

time: 623 ms


### Make a bar plot of the months in which movies whose titles start with "The Hobbit" are released in the USA.

In [13]:
release_dates[release_dates.title.rlike('^The Hobbit')]\
    .select(F.month(release_dates.date).alias('month'))\
    .groupby('month')\
    .count()\
    .sort('month')\
    .show()

+-----+-----+
|month|count|
+-----+-----+
|    1|    7|
|    2|    3|
|   11|    1|
|   12|  193|
+-----+-----+

time: 409 ms


### Make a bar plot of the day of the week on which movies with "Romance" in their title tend to be released in the USA.

In [16]:
release_dates[release_dates.title.rlike('Romance')]\
    .select(F.date_format(release_dates.date, 'E').alias('dayofweek'))\
    .groupby('dayofweek')\
    .count()\
    .sort('count')\
    .show()

+---------+-----+
|dayofweek|count|
+---------+-----+
|      Tue|   13|
|      Sat|   16|
|      Wed|   32|
|      Sun|   44|
|      Thu|   49|
|      Mon|   59|
|      Fri|   88|
+---------+-----+

time: 367 ms


### Make a bar plot of the day of the week on which movies with "Action" in their title tend to be released in the USA.

In [17]:
release_dates[release_dates.title.rlike('Action')]\
    .select(F.date_format(release_dates.date, 'E').alias('dayofweek'))\
    .groupby('dayofweek')\
    .count()\
    .sort('count')\
    .show()

+---------+-----+
|dayofweek|count|
+---------+-----+
|      Tue|    2|
|      Sun|    8|
|      Mon|   13|
|      Sat|   18|
|      Wed|   35|
|      Thu|   48|
|      Fri|  108|
+---------+-----+

time: 404 ms


### On which date was each Judi Dench movie from the 1990s released in the USA?

In [26]:
cast[(cast.name == 'Judi Dench') & (cast.year.between(1990, 1999))]\
    .select('title', 'year')\
    .join(release_dates, how='inner', on=['title', 'year'])\
    .filter("country = 'USA'")\
    .show()

+--------------------+----+-------+----------+
|               title|year|country|      date|
+--------------------+----+-------+----------+
|        Jack & Sarah|1995|    USA|1996-03-22|
| Tomorrow Never Dies|1997|    USA|1997-12-19|
|              Hamlet|1996|    USA|1996-12-25|
|           GoldenEye|1995|    USA|1995-11-17|
|The World Is Not ...|1999|    USA|1999-11-19|
| Shakespeare in Love|1998|    USA|1999-01-08|
|           Mrs Brown|1997|    USA|1997-07-18|
|  Tea with Mussolini|1999|    USA|1999-05-14|
+--------------------+----+-------+----------+

time: 754 ms


### In which months do films with Judi Dench tend to be released in the USA?

In [25]:
cast[(cast.name == 'Judi Dench')]\
    .select('title', 'year')\
    .join(release_dates, how='inner', on=['title', 'year'])\
    .filter("country = 'USA'")\
    .select(F.month('date').alias('month'))\
    .groupby('month')\
    .count()\
    .sort('month')\
    .show()

+-----+-----+
|month|count|
+-----+-----+
|    1|    4|
|    2|    3|
|    3|    4|
|    4|    2|
|    5|    4|
|    6|    3|
|    7|    2|
|    8|    1|
|   11|   10|
|   12|    4|
+-----+-----+

time: 1.87 s


### In which months do films with Tom Cruise tend to be released in the USA?

In [27]:
cast[(cast.name == 'Tom Cruise')]\
    .select('title', 'year')\
    .join(release_dates, how='inner', on=['title', 'year'])\
    .filter("country = 'USA'")\
    .select(F.month('date').alias('month'))\
    .groupby('month')\
    .count()\
    .sort('month')\
    .show()

+-----+-----+
|month|count|
+-----+-----+
|    1|    3|
|    3|    1|
|    4|    3|
|    5|    5|
|    6|    7|
|    7|    6|
|    8|    3|
|   10|    2|
|   11|    3|
|   12|    9|
+-----+-----+

time: 2.12 s
